# ESO 137-001 Image Depth

Calculates the $3\sigma$ and $5\sigma$ image depth of the *HST* ESO 137-001 images.

## Imports

In [1]:
# Python Imports
import warnings
from pathlib import Path

# Numerical Imports
import numpy as np

# Astropy Imports
from astropy import units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.convolution import convolve
from astropy.utils.exceptions import AstropyUserWarning
from regions import Regions
from photutils.background import Background2D
from photutils.segmentation import SourceFinder, make_2dgaussian_kernel
from photutils.utils import ImageDepth

## Notebook Setup

In [2]:
# Turn off Astropy Warnings in Photutils
warnings.filterwarnings('ignore', category=AstropyUserWarning)

In [5]:
# Constants
APER_RADIUS = {  # pix
    275: 0.55/0.03,
    475: 0.50/0.03,
    814: 0.50/0.03,
    160: 0.80/0.03
}

# N-Sigma
N_SIGMA = 1.5

In [6]:
# Image Paths
IMG_FNS = {
    275: Path('DrizzledImages/ESO137-001-F275W_drc.fits'),
    475: Path('DrizzledImages/ESO137-001-F475W_drc.fits'),
    814: Path('DrizzledImages/ESO137-001-F814W_drc.fits'),
    160: Path('DrizzledImages/ESO137-001-F160W_drz.fits')
}

# Image FOVs Paths
FOV_FNS = {
    275: Path('DS9/FOVs/WFC3-UVIS-FOV.reg'),
    475: Path('DS9/FOVs/ACS-WFC-FOV.reg'),
    814: Path('DS9/FOVs/ACS-WFC-FOV.reg'),
    160: Path('DS9/FOVs/WFC3-IR-FOV.reg')
}

In [7]:
# Zeropoint Values
ZPTS = {
    275: 24.171335,
    475: 26.060433,
    814: 25.948921,
    160: 25.946227
}

# Galactic Extinction Values
GAL_EXT = {
    275: 1.158,
    475: 0.689,
    814: 0.322,
    160: 0.108
}

## Calculate Image Depth

In [8]:
# Get Galaxy Footprint
galaxyRegions = Regions().read('DS9/GalaxyFootprint-BG_Calc.reg')

In [9]:
# Loop through Images
magLims, depth = {}, {}
galMask = None  # Initialize to avoid reference before assignment error
for key, fileName in IMG_FNS.items():

    # Read in Image
    with fits.open(fileName) as hdul:
        img = hdul['SCI'].data
        hdr = hdul['SCI'].header

    # Get Galaxy Mask
    if galMask is None:
        galMask = np.zeros_like(img, dtype=bool)  # Initialize if not set
        for reg in galaxyRegions:
            galMask |= reg.to_pixel(WCS(hdr)).to_mask().to_image(img.shape, bool)

    # Get the Coverage Mask
    coverageMask = Regions().read(FOV_FNS[key])[0].to_pixel(WCS(hdr)).to_mask()
    coverageMask = ~coverageMask.to_image(img.shape, bool)
    badPixMask   = ~np.isfinite(img) & ~coverageMask

    # Get Background
    bkg = Background2D(
        img, 128, mask=badPixMask, coverage_mask=coverageMask
    )

    # Set Threshold and Kernel
    if key != 160:
        kernel = make_2dgaussian_kernel(3.5*5/3, 9)
    else:
        kernel = make_2dgaussian_kernel(7, 11)
    threshold = 1.25 * bkg.background_rms

    # Get the Background Subtracted Image & Convolve
    img -= bkg.background
    convData = convolve(
        img, kernel
    )

    # Find Sources
    finder = SourceFinder(10, nproc=4, deblend=False)
    segMap = finder(convData, threshold)
    srcMask = segMap.make_source_mask()

    # Get the Depth
    mask_pad = 20 if key != 275 else 0
    depth[key] = ImageDepth(
        APER_RADIUS[key], nsigma=N_SIGMA, mask_pad=mask_pad, napers=1000, niters=50,
        overlap=False, seed=None, zeropoint=ZPTS[key] - GAL_EXT[key]
    )
    _, magLims[key] = depth[key](img, coverageMask | badPixMask | srcMask | galMask)


Image Depths:   0%|          | 0/50 [00:00<?, ?it/s]

Image Depths:   0%|          | 0/50 [00:00<?, ?it/s]

Image Depths:   0%|          | 0/50 [00:00<?, ?it/s]

Image Depths:   0%|          | 0/50 [00:00<?, ?it/s]

In [10]:
for key, magLim in magLims.items():
    print(f"F{key}W: {magLim:.2f} +/- {depth[key].mag_limits.std()<<u.ABmag:.2f} (image {N_SIGMA:.1f}-sigma depth)")

F275W: 25.33 +/- 0.03 mag(AB) (image 1.5-sigma depth)
F475W: 26.53 +/- 0.03 mag(AB) (image 1.5-sigma depth)
F814W: 25.96 +/- 0.03 mag(AB) (image 1.5-sigma depth)
F160W: 25.57 +/- 0.05 mag(AB) (image 1.5-sigma depth)


In [11]:
for key, d in depth.items():
    with open(f"DS9/Depths/ESO137-001-F{key}W-DepthApers.reg", "w") as fid:
        fid.write("# Region file format: DS9 version 4.1\n")
        fid.write(f"global color=cyan width=2\n")
        fid.write("image\n")
        for aper in d.apertures[0]:
            x, y = aper.positions
            r = aper.r
            fid.write(f"circle({x:.4f},{y:.4f},{r:.2f})\n")

## Get the Measured Data / Depth

In [ ]:
# Load Data
catalogs, mags = {}, {}
catPath = Path('/home/wwaldron/Research/Galaxies/ESO/ESO137001/Data/SExtractor/Catalogs')
for key in depth:

    # Use Astropy to Read SExtractor Catalog
    catalogs[key] = Table.read(catPath / f'ESO_F{key}WxF{key}W.cat', format='ascii.sextractor')


/home/wwaldron/Software/miniforge3/envs/stenv/lib/python3.12/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: invalid value encountered in log10
  return dex.to(self._function_unit, np.log10(x))
/home/wwaldron/Software/miniforge3/envs/stenv/lib/python3.12/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: invalid value encountered in log10
  return dex.to(self._function_unit, np.log10(x))
/home/wwaldron/Software/miniforge3/envs/stenv/lib/python3.12/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: invalid value encountered in log10
  return dex.to(self._function_unit, np.log10(x))
/home/wwaldron/Software/miniforge3/envs/stenv/lib/python3.12/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: invalid value encountered in log10
  return dex.to(self._function_unit, np.log10(x))


In [ ]:
# Keep the Sources Detected in Lower Three Bands
keepInds = {}
maxSep = 0.2 * u.arcsec
for key, cat in catalogs.items():

    # Assume Everything Detected
    keepInd = np.ones_like(cat['NUMBER'], dtype=bool)
    crds    = SkyCoord(
        ra= cat['X_WORLD'],
        dec=cat['Y_WORLD'],
        frame='fk5'
    )

    # Loop through Detection Bands
    for corrKey in [275, 475, 814]:

        # Skip Repeats
        if corrKey == key:
            continue

        # Get Coords from Catalog to Correlate
        corrCrds = SkyCoord(
            ra=catalogs[corrKey]['X_WORLD'],
            dec=catalogs[corrKey]['Y_WORLD'],
            frame='fk5'
        )

        # Get Matches
        _, d2d, _ = crds.match_to_catalog_sky(corrCrds)
        keepInd &= (d2d < maxSep)

    # Store Inds to Keep
    keepInds[key] = keepInd

In [178]:
# Trim Catalogs & Get Mags
for key, cat in catalogs.items():

    # Trim
    catalogs[key] = cat[keepInds[key]]

    # Get Mags
    mags[key]  = u.Magnitude(catalogs[key]['FLUX_APER'].value << u.electron/u.s)
    mags[key] -= u.Magnitude(GAL_EXT[key])
    mags[key] += u.Magnitude(ZPTS[key], 'mag(AB s / electron)')  # Apply Zeropoint

/home/wwaldron/Software/miniforge3/envs/stenv/lib/python3.12/site-packages/astropy/units/function/logarithmic.py:67: RuntimeWarning: invalid value encountered in log10
  return dex.to(self._function_unit, np.log10(x))


In [ ]:
for key, filtMags in mags.items():

    # Get Good Inds
    goodMask = np.isfinite(filtMags)

    # Print 3sigma Depths
    empDepth = filtMags[goodMask].max()
    print(f'{key} Faintest Source: {empDepth:.2f} (source {N_SIGMA:.1f}-sigma depth)')

275 Faintest Source: 28.33 mag(AB) (source 1.5-sigma depth)
475 Faintest Source: 26.65 mag(AB) (source 1.5-sigma depth)
814 Faintest Source: 26.59 mag(AB) (source 1.5-sigma depth)
160 Faintest Source: 25.41 mag(AB) (source 1.5-sigma depth)
